# How to Make API Calls Using Python and the AWS Boto3 Library

In this notebook, we'll use the boto3 Python SDK to interact with Amazon Bedrock Foundation Models.

1) Install the required dependencies to interact with Amazon Bedrock

2) Use the Amazon Bedrock API to initialize the Bedrock client and verify connectivity

3) Invoke the Bedrock API and display the results

## 1. Install the required dependencies to interact with Amazon Bedrock

Install the required packages needed. Ignore any pip dependency errors, they won't affect what we're doing.

In [ ]:
%pip install --upgrade -q botocore
%pip install --upgrade -q boto3
%pip install --upgrade -q awscli

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## 2. Use the Amazon Bedrock API to initialize the Bedrock client and verify connectivity
Interaction with the Bedrock API is done using boto3 the AWS SDK for Python.

In [ ]:
import json
import os
import sys

import boto3

boto3_bedrock = boto3.client('bedrock')

### Verify connectivity
We can check the client works by trying out the list_foundation_models() method, this will display all the models available for us to use

In [ ]:
boto3_bedrock.list_foundation_models()

## 3. Invoke the Bedrock API and display the results
Run the cells below to invoke the Bedrock runtime used to interact with models

In [ ]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')

### Build our prompt
Edit this cell to use a different prompt

In [ ]:
prompt_data = """Command: Write a social post about cost optimization on AWS

Post:
"""

### Invoke the model using the prompt
Construct the body using the prompt_data from the previous cell
Add parameters like topP and temperature to control the level creativity

In [ ]:
try:

    body = json.dumps({"inputText": prompt_data, "textGenerationConfig" : {"topP":0.95, "temperature":0.2}})
    modelId = "amazon.titan-text-lite-v1"
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("results")[0].get("outputText"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Create an image using Stability Stable Diffusion XL
## Build our prompt
Edit this cell to use a different prompt

In [ ]:
prompt_data = "a rocket with AWS printed on the side"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl-v1"
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


### Decode the image using PIL 
The output from the model is a base64 encoded string of the image data. 
Use an image processing library like Pillow (PIL) to decode the image so we can view it.

In [ ]:
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
image